In [2]:
from grammarLR1 import GrammarLR1
from grammarLR0 import GrammarLR0
from common.token_class import Token

In [3]:
def gramophoneSyntaxParser( inputTokens : str) -> GrammarLR1:
    non_terminals = []
    terminals = []
    start_symbol = ""
    productions = {}
    
    def add_symbol( symbol : str):
        symbol = symbol.strip()
        if len(symbol)  == 0:
            pass
        elif symbol[0].islower():
            if symbol not in terminals:
                terminals.append(symbol)
        else:
            if symbol not in non_terminals:
                non_terminals.append(symbol)
                productions[symbol] = []
        return symbol


    lines = inputTokens.split("\n")
    for line in lines:
        if len(line) > 0:
            line = line.strip() 
            line = line[:-1]
            parts = line.split("->")
            non_terminal = add_symbol(parts[0])
            if start_symbol == "":
                start_symbol = non_terminal
            productionsLine = parts[1].split("|")
            for prod in productionsLine:
                prod = prod.strip().split(" ")
                prod_to_add = []
                for symbol in prod:
                    symbol = add_symbol(symbol)
                    if len(symbol) > 0:
                        prod_to_add.append(symbol)
                productions[non_terminal].append(prod_to_add)

    return GrammarLR1(non_terminals, terminals, start_symbol, productions)


In [3]:
inputTokensAlvaro = """

Program -> ListDeclaration ExprSemicolon .

ListDeclaration -> Declaration ListDeclaration | .

Declaration -> ClassDecl | FunctionDecl | ProtocolDecl.

ClassDecl -> type id TypeBody .

TypeBody -> lbrace ListTypeBodyItems rbrace . 

ListTypeBodyItems -> TypeBodyItems ListTypeBodyItems | .

TypeBodyItems -> PrimitiveAssignment semicolon | MethodDecl.

PrimitiveAssignment -> id assign Expr .

MethodDeclFullForm -> id ArgNamesList BlockExpr .

MethodDeclInline ->  id ArgNamesList arrowright SingleExprSemicolon .

MethodDecl -> MethodDeclFullForm | MethodDeclInline .

FunctionDecl -> function MethodDecl.

ArgNamesList -> lparen BlockArgNamesList rparen .

BlockArgNamesList -> NonEmptyArgNamesList | .

NonEmptyArgNamesList -> id | id comma NonEmptyArgNamesList .

SingleExprSemicolon -> SingleExpr semicolon .

BlockExprSemicolon  ->   BlockExpr OptionalSemicolon.

OptionalSemicolon -> semicolon | .

ExprSemicolon -> SingleExprSemicolon | BlockExprSemicolon.
Expr -> SingleExpr | BlockExpr.

BlockExpr -> lbrace ExprBlock rbrace .

ExprBlock -> ExprSemicolon ExprBlock | . 

SingleExpr -> IfExpr | ForExpr | WhileExpr | LetExpr | DestructiveExpr | Term .

BodyExpr -> BlockExpr | Term .

IfExpr -> if lparen Expr rparen BodyExpr OptionalElif else BodyExpr .

OptionalElif -> elif lparen Expr rparen BodyExpr OptionalElif | . 

WhileExpr -> while lparen Expr rparen BodyExpr .

ForExpr -> for lparen id in BodyExpr rparen BodyExpr .

LetExpr -> let AsignmentLetList in Expr.

DestructiveExpr -> id destructiveoperator Expr .

AsignmentLetList -> PrimitiveAssignment | AsignmentLetListElements.

AsignmentLetListElements -> comma AsignmentLetList | .

Term -> Three .

Three -> ThreeLeft | ThreeRight .

ThreeRight -> Two | Two ThreeRightOperator ThreeRight.

ThreeRightOperator -> equal | notequal | greater | less | greaterequal | lessequal .

Two -> TwoLeft | TwoRight .

TwoLeft -> One | TwoLeft TwoLeftOperator One .

TwoLeftOperator -> plus | minus | and .

One -> OneLeft | OneRight . 

OneRight -> Zero | Zero OneRightOperator OneRight .

OneRightOperator -> mult | division | or .

Zero -> ZeroLeft | ZeroRight.

ZeroRight -> UnaryExpr | UnaryExpr ZeroRightOperator ZeroRight .

ZeroRightOperator -> exponentiation.

UnaryExpr -> DottedBase | UnaryOperator Base .

UnaryOperator -> minus | exclamation .

DottedBase -> Base | DottedBase dot Base . 

Base -> FunctionCall | lparen SingleExpr rparen | Literal.

Literal -> id | number | string | boolean .

FunctionCall -> Base ArgCallList .

ArgCallList -> lparen ArgsList rparen .

ArgsList -> NonEmptyArgCallList | .

NonEmptyArgCallList -> SingleExpr | SingleExpr comma NonEmptyArgCallList .

"""

grAlvaro = gramophoneSyntaxParser(inputTokensAlvaro)

In [4]:
tableAlvaro = grAlvaro.build_parsing_table()

print(tableAlvaro)

State                                           type                                            id                                              lbrace                                          rbrace                                          semicolon                                       assign                                          arrowright                                      function                                        lparen                                          rparen                                          comma                                           if                                              else                                            elif                                            while                                           for                                             in                                              let                                             destructiveoperator                             equal                                   

In [5]:
# 4;

node = tableAlvaro.parse(
    [Token(x, x, 0, 0) for x in ["lparen", "number", "rparen", "semicolon", "$"]]
)

node.root.print([0], 0, True)

for nonterminal in grAlvaro.non_terminals:
    print(nonterminal,  grAlvaro.productions[nonterminal])







Program:Program
|
|_______ListDeclaration:ListDeclaration
|
|_______ExprSemicolon:ExprSemicolon
        |
        |_______SingleExprSemicolon:SingleExprSemicolon
                |
                |_______SingleExpr:SingleExpr
                |       |
                |       |_______Term:Term
                |               |
                |               |_______Three:Three
                |                       |
                |                       |_______ThreeRight:ThreeRight
                |                               |
                |                               |_______Two:Two
                |                                       |
                |                                       |_______TwoLeft:TwoLeft
                |                                               |
                |                                               |_______One:One
                |                                                       |
                |                  

In [6]:
# { 
#   { 
#       4; 
#   } 
# };

node = tableAlvaro.parse(
    [Token(x, x, 0, 0) for x in ["lbrace", "number", "semicolon", "rbrace", "$"]]
)

node.root.print([0], 0, True)


Program:Program
|
|_______ListDeclaration:ListDeclaration
|
|_______ExprSemicolon:ExprSemicolon
        |
        |_______BlockExprSemicolon:BlockExprSemicolon
                |
                |_______BlockExpr:BlockExpr
                |       |
                |       |_______lbrace:lbrace
                |       |
                |       |_______ExprBlock:ExprBlock
                |       |       |
                |       |       |_______ExprSemicolon:ExprSemicolon
                |       |       |       |
                |       |       |       |_______SingleExprSemicolon:SingleExprSemicolon
                |       |       |               |
                |       |       |               |_______SingleExpr:SingleExpr
                |       |       |               |       |
                |       |       |               |       |_______Term:Term
                |       |       |               |               |
                |       |       |               |               |____

In [4]:
inputTokensJavier = """

Program -> Decls Expr semicolon.
Decls -> FuncDecl Decls | TypeDecl Decls | ProtocolDecl Decls | .

ProtocolDecl -> protocol id OptExtension lbrace ProtocolElems rbrace.
OptExtension -> extends id | .
ProtocolElems -> MethodSignature ProtocolElems | .
MethodSignature -> id lparen TypedParamList rparen colon id semicolon.
TypedParamList -> id colon id TypedParamTail | .
TypedParamTail -> comma id colon id TypedParamTail | .

TypeDecl -> type id OptParams OptInheritance lbrace TypeElems rbrace .

OptParams -> lparen ParamList rparen | .
OptInheritance -> inherits id OptArgs | .
OptArgs -> lparen ArgList rparen | .

TypeElems -> AttributeDecl TypeElems | MethodDecl TypeElems | .
AttributeDecl -> Assignment  semicolon .
MethodDecl -> id lparen ParamList rparen OptType FuncBody .
ParamList -> id OptType ParamTail | .
ParamTail -> comma id OptType ParamTail | .
FuncBody -> arrow Expr semicolon | BlockExpr semicolon .

Assignment -> id OptType equal Expr .
OptType -> colon id | .

FuncDecl -> function MethodDecl .

Expr -> BlockExpr | IfExpr | WhileExpr | ForExpr | LetExpr | DestrucExpr | VectorExpr | NewExpr | As .

LetExpr -> let Assignment AssignmentList in Expr .
AssignmentList -> comma Assignment AssignmentList | .

BlockExpr -> lbrace ExprList rbrace .
ExprList -> Expr semicolon ExprTail .
ExprTail -> Expr semicolon ExprTail | .

IfExpr -> if lparen Expr rparen Expr OptElif else Expr .
OptElif -> elif lparen Expr rparen Expr OptElif | .

WhileExpr -> while lparen Expr rparen Expr .
ForExpr -> for lparen id in Expr rparen Expr .

DestrucExpr -> Primary destrucOp Expr .

VectorExpr -> lbracket VectorElems rbracket | lbracket Expr doubleOr id in Expr rbracket .
VectorElems -> Expr VectorTail | .
VectorTail -> comma Expr VectorTail | .

NewExpr -> new CallList .

As -> LogicOr asOp id | LogicOr .
LogicOr -> LogicOr or LogicAnd | LogicAnd .
LogicAnd -> LogicAnd and Equality | Equality .
Equality -> Equality doubleEqual Comparison | Equality notEqual Comparison | Comparison .
Comparison -> Comparison greater Is | Comparison greaterEq Is | Comparison less Is | Comparison lessEq Is | Is .
Is -> Is isOp Str | Str .
Str -> Str StrOp Term | Term .
StrOp -> at | doubleAt .
Term -> Term plus Factor | Term minus Factor | Factor .
Factor -> Factor star Mod | Factor div Mod | Mod .
Mod -> Mod modOp Power | Power .
Power -> Primary powerOp number | Primary .

Primary -> false | true | number | string | CallList .

CallList -> CallList dot id | CallList lparen ArgList rparen | CallList lbracket Expr rbracket | self | id | lparen Expr rparen .
ArgList -> Expr ArgTail | .
ArgTail -> comma Expr ArgTail | .

"""

grJavier = gramophoneSyntaxParser(inputTokensJavier)




In [5]:
tableJavier = grJavier.build_parsing_table()
# print(tableJavier)

### Test 1

In [6]:
node = tableJavier.parse(
    [Token(x, x, 0, 0) for x in [
        'lbrace',
        'let', 'id', 'colon', 'id', 'equal', 'id', 'plus', 'number', 'in', 'lparen', 'number', 'rparen', 'semicolon',
        
        'id', 'destrucOp', 'lparen', 'number', 'plus', 'number', 'rparen', 'semicolon',

        'number', 'semicolon',

        'id', 'dot', 'id', 'semicolon',

        'id', 'dot', 'id', 'dot', 'id', 'lparen', 'rparen', 'semicolon',
        'rbrace', 'semicolon',
        '$']]
)

node.root.print([0], 0, True)


Program:Program
|
|_______Decls:Decls
|
|_______Expr:Expr
|       |
|       |_______BlockExpr:BlockExpr
|               |
|               |_______lbrace:lbrace
|               |
|               |_______ExprList:ExprList
|               |       |
|               |       |_______Expr:Expr
|               |       |       |
|               |       |       |_______LetExpr:LetExpr
|               |       |               |
|               |       |               |_______let:let
|               |       |               |
|               |       |               |_______Assignment:Assignment
|               |       |               |       |
|               |       |               |       |_______id:id
|               |       |               |       |
|               |       |               |       |_______OptType:OptType
|               |       |               |       |       |
|               |       |               |       |       |_______colon:colon
|               |       |               |  

### Test 2

In [7]:
node = tableJavier.parse(
    [Token(x, x, 0, 0) for x in [
        'lbrace',
        'id', 'dot', 'id', 'dot', 'id', 'lparen', 'number', 'comma', 'id', 'comma', 'id', 'lparen', 'rparen', 'rparen', 'semicolon',
        
        'id', 'lbracket', 'number', 'rbracket', 'dot', 'id', 'lparen', 'rparen', 'semicolon',

        'number', 'powerOp', 'number', 'plus', 'id', 'lparen', 'rparen', 'semicolon',

        'lbrace', 
            'id', 'destrucOp', 'lparen', 'let', 'id', 'colon', 'id', 'equal', 'id', 'plus', 'number', 'in', 'lparen', 'number', 'rparen', 'rparen', 'semicolon'
        ,'rbrace', 'semicolon',
        'rbrace', 'semicolon',
        '$']]
)

node.root.print([0], 0, True)


Program:Program
|
|_______Decls:Decls
|
|_______Expr:Expr
|       |
|       |_______BlockExpr:BlockExpr
|               |
|               |_______lbrace:lbrace
|               |
|               |_______ExprList:ExprList
|               |       |
|               |       |_______Expr:Expr
|               |       |       |
|               |       |       |_______As:As
|               |       |               |
|               |       |               |_______LogicOr:LogicOr
|               |       |                       |
|               |       |                       |_______LogicAnd:LogicAnd
|               |       |                               |
|               |       |                               |_______Equality:Equality
|               |       |                                       |
|               |       |                                       |_______Comparison:Comparison
|               |       |                                               |
|               |       |  

### Test 3

In [8]:
node = tableJavier.parse(
    [Token(x, x, 0, 0) for x in [
        'lbrace',
        'while', 'lparen', 'true', 'rparen', 'number', 'semicolon', 

        'while', 'lparen', 'while', 'lparen', 'false', 'rparen', 'lbrace', 'id', 'semicolon', 'rbrace', 'rparen', 'number', 'semicolon',

        'if', 'lparen', 'id', 'doubleEqual', 'id', 'rparen', 
            'string',
        'else',
            'string',
        'semicolon',
        'rbrace', 'semicolon',
        '$']]
)

node.root.print([0], 0, True)


Program:Program
|
|_______Decls:Decls
|
|_______Expr:Expr
|       |
|       |_______BlockExpr:BlockExpr
|               |
|               |_______lbrace:lbrace
|               |
|               |_______ExprList:ExprList
|               |       |
|               |       |_______Expr:Expr
|               |       |       |
|               |       |       |_______WhileExpr:WhileExpr
|               |       |               |
|               |       |               |_______while:while
|               |       |               |
|               |       |               |_______lparen:lparen
|               |       |               |
|               |       |               |_______Expr:Expr
|               |       |               |       |
|               |       |               |       |_______As:As
|               |       |               |               |
|               |       |               |               |_______LogicOr:LogicOr
|               |       |               |                    

### Test 4

In [9]:
node = tableJavier.parse(
    [Token(x, x, 0, 0) for x in [
        'lbrace',
        'if', 'lparen', 'id', 'doubleEqual', 'id', 'rparen', 
            'string',
        'elif', 'lparen', 'number', 'greater', 'id', 'rparen',
            'id', 'lparen', 'rparen', 'lbracket', 'id', 'rbracket',
        'else',
            'string',
        'semicolon',

        'for', 'lparen', 'id', 'in', 'lbracket', 'number', 'comma', 'number', 'comma', 'number', 'comma', 'number', 'rbracket', 'rparen', 'id', 'lparen', 'id', 'comma', 'id','rparen', 'semicolon',
        'rbrace', 'semicolon',
        '$']]
)

node.root.print([0], 0, True)


Program:Program
|
|_______Decls:Decls
|
|_______Expr:Expr
|       |
|       |_______BlockExpr:BlockExpr
|               |
|               |_______lbrace:lbrace
|               |
|               |_______ExprList:ExprList
|               |       |
|               |       |_______Expr:Expr
|               |       |       |
|               |       |       |_______IfExpr:IfExpr
|               |       |               |
|               |       |               |_______if:if
|               |       |               |
|               |       |               |_______lparen:lparen
|               |       |               |
|               |       |               |_______Expr:Expr
|               |       |               |       |
|               |       |               |       |_______As:As
|               |       |               |               |
|               |       |               |               |_______LogicOr:LogicOr
|               |       |               |                       |
|      

### Test 5

In [11]:
node = tableJavier.parse(
    [Token(x, x, 0, 0) for x in [
        'function', 'id', 'lparen', 'id', 'rparen', 'arrow', 'id', 'lparen', 'id', 'rparen', 'div', 'id', 'lparen', 'id', 'rparen', 'semicolon',
        'function', 'id', 'lparen', 'id', 'comma', 'id', 'rparen', 
        'lbrace',
            'id', 'lparen', 'id', 'plus', 'id', 'rparen', 'semicolon',
            'id', 'lparen', 'id', 'minus', 'id', 'rparen', 'semicolon',
            'id', 'lparen', 'id', 'star', 'id', 'rparen', 'semicolon',
        'rbrace', 'semicolon', 

        'lbrace',

        'id', 'lparen', 'lparen', 'lparen', 'lparen', 'number', 'plus', 'number', 'rparen', 'powerOp', 'number', 'rparen', 'star', 'number', 'rparen', 'div', 'number' ,'rparen', 'semicolon',
        
        'id', 'lparen', 'string', 'at', 'number', 'rparen', 'semicolon',

        'lbrace', 
            'id', 'lparen', 'number', 'rparen', 'semicolon',
            'id', 'lparen', 'id', 'lparen', 'rparen', 'rparen', 'semicolon',
            'id', 'lparen', 'string', 'rparen', 'semicolon',
        'rbrace', 'semicolon',

        'id', 'lparen', 'id', 'lparen','rparen', 'powerOp', 'number', 'rparen', 'semicolon',
        'rbrace', 'semicolon',
        '$']]
)

node.root.print([0], 0, True)


Program:Program
|
|_______Decls:Decls
|       |
|       |_______FuncDecl:FuncDecl
|       |       |
|       |       |_______function:function
|       |       |
|       |       |_______MethodDecl:MethodDecl
|       |               |
|       |               |_______id:id
|       |               |
|       |               |_______lparen:lparen
|       |               |
|       |               |_______ParamList:ParamList
|       |               |       |
|       |               |       |_______id:id
|       |               |       |
|       |               |       |_______OptType:OptType
|       |               |       |
|       |               |       |_______ParamTail:ParamTail
|       |               |
|       |               |_______rparen:rparen
|       |               |
|       |               |_______OptType:OptType
|       |               |
|       |               |_______FuncBody:FuncBody
|       |                       |
|       |                       |_______arrow:arrow
|      

### Test 6

In [14]:
node = tableJavier.parse(
    [Token(x, x, 0, 0) for x in [
        'lbrace',

        'let', 'id', 'equal', 'string', 'in', 'id', 'lparen', 'string', 'rparen', 'semicolon',

        'let', 'id', 'equal', 'number', 'comma', 'id', 'equal', 'string', 'in', 'id', 'lparen', 'id', 'at', 'id', 'rparen', 'semicolon',

        'let', 'id', 'equal', 'number', 'in',
            'let', 'id', 'equal', 'string', 'in',
                'id', 'lparen', 'id', 'at', 'id', 'rparen', 'semicolon',

        'let', 'id', 'equal', 'number', 'comma', 'id', 'equal', 'number', 'comma', 'id', 'equal', 'number', 'in', 
        'lbrace', 
            'id', 'lparen', 'number', 'rparen', 'semicolon',
            'id', 'lparen', 'id', 'lparen', 'rparen', 'rparen', 'semicolon',
            'id', 'lparen', 'string', 'rparen', 'semicolon',
        'rbrace', 'semicolon',

        'let', 'id', 'equal', 'number', 'in', 
        'lbrace',
            'let', 'id', 'equal', 'number', 'in', 'id', 'lparen', 'id', 'rparen', 'semicolon',
        'rbrace', 'semicolon',
        'rbrace', 'semicolon',
        '$']]
)

node.root.print([0], 0, True)


Program:Program
|
|_______Decls:Decls
|
|_______Expr:Expr
|       |
|       |_______BlockExpr:BlockExpr
|               |
|               |_______lbrace:lbrace
|               |
|               |_______ExprList:ExprList
|               |       |
|               |       |_______Expr:Expr
|               |       |       |
|               |       |       |_______LetExpr:LetExpr
|               |       |               |
|               |       |               |_______let:let
|               |       |               |
|               |       |               |_______Assignment:Assignment
|               |       |               |       |
|               |       |               |       |_______id:id
|               |       |               |       |
|               |       |               |       |_______OptType:OptType
|               |       |               |       |
|               |       |               |       |_______equal:equal
|               |       |               |       |
|        

### Test 7

In [15]:
node = tableJavier.parse(
    [Token(x, x, 0, 0) for x in [
        'type', 'id', 
        'lbrace',
            'id', 'equal', 'number', 'semicolon',
            'id', 'equal', 'number', 'semicolon',

            'id', 'lparen', 'rparen', 'arrow', 'self', 'dot', 'id', 'semicolon',
            'id', 'lparen', 'rparen', 'arrow', 'self', 'dot', 'id', 'semicolon',

            'id', 'lparen', 'id', 'rparen', 'arrow', 'self', 'dot', 'id', 'destrucOp', 'id',  'semicolon',
            'id', 'lparen', 'id', 'rparen', 'arrow', 'self', 'dot', 'id', 'destrucOp', 'id',  'semicolon',
        'rbrace',

        'type', 'id', 'lparen', 'id', 'colon', 'id', 'comma', 'id','rparen', 
        'lbrace',
            'id', 'equal', 'number', 'semicolon',
            'id', 'equal', 'number', 'semicolon',

            'id', 'lparen', 'rparen', 'arrow', 'self', 'dot', 'id', 'semicolon',
            'id', 'lparen', 'rparen', 'arrow', 'self', 'dot', 'id', 'semicolon',

            'id', 'lparen', 'id', 'rparen', 'arrow', 'self', 'dot', 'id', 'destrucOp', 'id',  'semicolon',
            'id', 'lparen', 'id', 'rparen', 'arrow', 'self', 'dot', 'id', 'destrucOp', 'id',  'semicolon',
        'rbrace',

        'type', 'id', 'lparen', 'id', 'colon', 'id', 'comma', 'id','rparen', 'inherits', 'id',
        'lbrace',
            'id', 'equal', 'number', 'semicolon',
            'id', 'equal', 'number', 'semicolon',

            'id', 'lparen', 'rparen', 'arrow', 'self', 'dot', 'id', 'semicolon',
            'id', 'lparen', 'rparen', 'arrow', 'self', 'dot', 'id', 'semicolon',

            'id', 'lparen', 'id', 'rparen', 'arrow', 'self', 'dot', 'id', 'destrucOp', 'id',  'semicolon',
            'id', 'lparen', 'id', 'rparen', 'arrow', 'self', 'dot', 'id', 'destrucOp', 'id',  'semicolon',
        'rbrace',

        'type', 'id', 'lparen', 'id', 'colon', 'id', 'comma', 'id','rparen', 'inherits', 'id', 'lparen', 'id', 'comma', 'id' , 'rparen',
        'lbrace',
            'id', 'equal', 'number', 'semicolon',
            'id', 'equal', 'number', 'semicolon',

            'id', 'lparen', 'rparen', 'arrow', 'self', 'dot', 'id', 'semicolon',
            'id', 'lparen', 'rparen', 'arrow', 'self', 'dot', 'id', 'semicolon',

            'id', 'lparen', 'id', 'rparen', 'arrow', 'self', 'dot', 'id', 'destrucOp', 'id',  'semicolon',
            'id', 'lparen', 'id', 'rparen', 'arrow', 'self', 'dot', 'id', 'destrucOp', 'id',  'semicolon',
        'rbrace',

        'number', 'semicolon',
        '$']]
)

node.root.print([0], 0, True)


Program:Program
|
|_______Decls:Decls
|       |
|       |_______TypeDecl:TypeDecl
|       |       |
|       |       |_______type:type
|       |       |
|       |       |_______id:id
|       |       |
|       |       |_______OptParams:OptParams
|       |       |
|       |       |_______OptInheritance:OptInheritance
|       |       |
|       |       |_______lbrace:lbrace
|       |       |
|       |       |_______TypeElems:TypeElems
|       |       |       |
|       |       |       |_______AttributeDecl:AttributeDecl
|       |       |       |       |
|       |       |       |       |_______Assignment:Assignment
|       |       |       |       |       |
|       |       |       |       |       |_______id:id
|       |       |       |       |       |
|       |       |       |       |       |_______OptType:OptType
|       |       |       |       |       |
|       |       |       |       |       |_______equal:equal
|       |       |       |       |       |
|       |       |       |       |    

### Test 8

In [16]:
node = tableJavier.parse(
    [Token(x, x, 0, 0) for x in [
        'lbrace',
        'let', 'id', 'equal', 'number', 'in', 'id', 'lparen', 'if', 'lparen', 'id', 'modOp', 'number', 'doubleEqual', 'number', 'rparen', 'number', 'else', 'number','rparen', 'semicolon',
        
        'let', 'id', 'equal', 'number', 'in',
            'if', 'lparen', 'id', 'modOp', 'number', 'doubleEqual', 'number', 'rparen',
            'lbrace',
                'id', 'lparen', 'id', 'rparen', 'semicolon',
                'id', 'lparen', 'string', 'rparen', 'semicolon',
            'rbrace',
            'else', 'id', 'lparen', 'string', 'rparen', 'semicolon',
        'rbrace', 'semicolon',
        '$']]
)

node.root.print([0], 0, True)


Program:Program
|
|_______Decls:Decls
|
|_______Expr:Expr
|       |
|       |_______BlockExpr:BlockExpr
|               |
|               |_______lbrace:lbrace
|               |
|               |_______ExprList:ExprList
|               |       |
|               |       |_______Expr:Expr
|               |       |       |
|               |       |       |_______LetExpr:LetExpr
|               |       |               |
|               |       |               |_______let:let
|               |       |               |
|               |       |               |_______Assignment:Assignment
|               |       |               |       |
|               |       |               |       |_______id:id
|               |       |               |       |
|               |       |               |       |_______OptType:OptType
|               |       |               |       |
|               |       |               |       |_______equal:equal
|               |       |               |       |
|        

### Test 9

In [17]:
node = tableJavier.parse(
    [Token(x, x, 0, 0) for x in [
        'function', 'id', 'lparen', 'id', 'comma', 'id', 'rparen', 'arrow', 'while', 'lparen', 'id', 'greater', 'number', 'rparen',
        'let', 'id', 'equal', 'id', 'modOp', 'id', 'in',
        'lbrace',
            'id', 'destrucOp', 'id', 'semicolon',
            'id', 'destrucOp', 'id', 'semicolon',
        'rbrace', 'semicolon',
        
        'let', 'id', 'equal', 'id', 'lparen', 'number', 'comma', 'number', 'rparen', 'in',
            'while', 'lparen', 'id', 'dot', 'id', 'lparen', 'rparen', 'rparen',
                'let', 'id', 'equal', 'id', 'dot', 'id', 'lparen', 'rparen', 'in', 
                    'id', 'lparen', 'id', 'rparen', 'semicolon',
        '$']]
)

node.root.print([0], 0, True)


Program:Program
|
|_______Decls:Decls
|       |
|       |_______FuncDecl:FuncDecl
|       |       |
|       |       |_______function:function
|       |       |
|       |       |_______MethodDecl:MethodDecl
|       |               |
|       |               |_______id:id
|       |               |
|       |               |_______lparen:lparen
|       |               |
|       |               |_______ParamList:ParamList
|       |               |       |
|       |               |       |_______id:id
|       |               |       |
|       |               |       |_______OptType:OptType
|       |               |       |
|       |               |       |_______ParamTail:ParamTail
|       |               |               |
|       |               |               |_______comma:comma
|       |               |               |
|       |               |               |_______id:id
|       |               |               |
|       |               |               |_______OptType:OptType
|       |  

### Test 10

In [21]:
node = tableJavier.parse(
    [Token(x, x, 0, 0) for x in [
        'type', 'id', 'lparen', 'id', 'colon', 'id', 'comma', 'id', 'colon', 'id', 'rparen',
        'lbrace',
            'id', 'equal', 'id', 'semicolon',
            'id', 'equal', 'id', 'semicolon',
            'id', 'equal', 'id', 'minus', 'number','semicolon',

            'id', 'lparen', 'rparen', 'colon', 'id', 'arrow', 'lparen', 'self', 'dot', 'id', 'destrucOp', 'self', 'dot', 'id', 'plus', 'number', 'rparen', 'less', 'id', 'semicolon',
        'rbrace',

        'protocol', 'id',
        'lbrace',
            'id', 'lparen', 'id', 'colon', 'id', 'rparen', 'colon', 'id', 'semicolon',
            'id', 'lparen', 'rparen', 'colon', 'id', 'semicolon',
        'rbrace',
        
        'protocol', 'id', 'extends', 'id',
        'lbrace',
            'id', 'lparen', 'id', 'colon', 'id', 'rparen', 'colon', 'id', 'semicolon',
            'id', 'lparen', 'rparen', 'colon', 'id', 'semicolon',
        'rbrace',

        'lbrace',

        'let', 'id', 'equal', 'lbracket', 'id', 'doubleOr', 'id', 'in', 'id', 'lparen', 'id', 'comma', 'id', 'rparen', 'rbracket', 'in', 'id', 'lparen', 'id', 'rparen', 'semicolon',

        'let', 'id', 'equal', 'lbracket', 'number', 'comma', 'number', 'comma', 'number', 'comma', 'number', 'comma', 'number', 'rbracket', 'in',
        'for', 'lparen', 'id', 'in', 'id', 'rparen',
            'id', 'lparen', 'id', 'rparen', 'semicolon',

        'lparen', 'id', 'rparen', 'dot', 'id', 'semicolon',

        'let', 'id', 'equal', 'new', 'id', 'lparen', 'number', 'comma', 'id', 'rparen', 'in', 'id', 'lparen', 'id', 'rparen', 'semicolon',
        'rbrace', 'semicolon',
        '$']]
)

node.root.print([0], 0, True)


Program:Program
|
|_______Decls:Decls
|       |
|       |_______TypeDecl:TypeDecl
|       |       |
|       |       |_______type:type
|       |       |
|       |       |_______id:id
|       |       |
|       |       |_______OptParams:OptParams
|       |       |       |
|       |       |       |_______lparen:lparen
|       |       |       |
|       |       |       |_______ParamList:ParamList
|       |       |       |       |
|       |       |       |       |_______id:id
|       |       |       |       |
|       |       |       |       |_______OptType:OptType
|       |       |       |       |       |
|       |       |       |       |       |_______colon:colon
|       |       |       |       |       |
|       |       |       |       |       |_______id:id
|       |       |       |       |
|       |       |       |       |_______ParamTail:ParamTail
|       |       |       |               |
|       |       |       |               |_______comma:comma
|       |       |       |               |